# Text Clustering and Topic Modeling

## A Common Pipeline for Text Clustering

#### Although there are many methods for text clustering, from graph-based neural networks to centroid-based clustering techniques, a common pipeline that has gained popularity involves three steps and algorithms:

- Convert the input documents to embeddings with an embedding model.

- Reduce the dimensionality of embeddings with a dimensionality reduction model.

- Find groups of semantically similar documents with a cluster model.

In [1]:
# Load data from Hugging Face
from datasets import load_dataset
dataset = load_dataset("maartengr/arxiv_nlp")["train"]

# Extract metadata
abstracts = dataset["Abstracts"]
titles = dataset["Titles"]

from sentence_transformers import SentenceTransformer

# Create an embedding for each abstract
embedding_model = SentenceTransformer("thenlper/gte-small")
embeddings = embedding_model.encode(abstracts, show_progress_bar=True)

ModuleNotFoundError: No module named 'sentence_transformers'